In [8]:
# =========================================================
# LightGBM Regressor with SMAPE
# =========================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore")

# =========================================================
# SMAPE metric
# =========================================================
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0
    return np.mean(diff) * 100

# =========================================================
# Load Data
# =========================================================
data = pd.read_csv("final_pca_whole_train_data.csv")
X = data.drop(columns=["price", "image_link", "sample_id"])
y = data["price"].values

# =========================================================
# Train / Validation Split
# =========================================================
X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# =========================================================
# Scale features
# =========================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_val_scaled = scaler.transform(X_val_raw)

# =========================================================
# Log-transform target
# =========================================================
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# =========================================================
# LightGBM model (scikit-learn API)
# =========================================================
lgb_model = lgb.LGBMRegressor(
    objective="regression",
    learning_rate=0.05,
    n_estimators=150,
    num_leaves=32,
    feature_fraction=0.9,
    bagging_fraction=0.95,
    bagging_freq=3,
    random_state=42,
    n_jobs=-1,

)

# =========================================================
# Fit with early stopping
# =========================================================
print("Training LightGBM...")
lgb_model.fit(
    X_train_scaled, y_train_log,
    eval_set=[(X_val_scaled, y_val_log)],
    eval_metric="l2",
)

# =========================================================
# Predict and evaluate
# =========================================================
y_pred_log = lgb_model.predict(X_val_scaled)
y_pred = np.expm1(y_pred_log)

mae = mean_absolute_error(y_val, y_pred)
smape_val = smape(y_val, y_pred)

print("==========================")
print(f"Final MAE: {mae:.4f}")
print(f"Final SMAPE: {smape_val:.2f}%")
print("==========================")

Training LightGBM...
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.557147 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 153261
[LightGBM] [Info] Number of data points in the train set: 59247, number of used

In [10]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   ------------------------------ --------- 3/4 [optuna]
   ------------------------------ --------- 3/4 [optuna]
   ------------------------------ --------- 3/4 [optuna]
   ------------------------------ --------- 3/4 [optuna]
   ---------

In [18]:
# =========================================================
# LightGBM Regressor with SMAPE + Optuna Tuning (v4 - Final)
# =========================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
import optuna
import lightgbm as lgb
from optuna.integration import LightGBMTunerCV
import warnings

warnings.filterwarnings("ignore")

# =========================================================
# SMAPE metric
# =========================================================
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff) * 100

# =========================================================
# Load Data
# =========================================================
try:
    data = pd.read_csv("final_pca_whole_train_data.csv")
except FileNotFoundError:
    print("Error: Data file not found. Creating a dummy dataframe for demonstration.")
    data = pd.DataFrame({
        'price': np.random.lognormal(mean=8, sigma=1, size=500),
        'feature1': np.random.rand(500) * 10,
        'feature2': np.random.rand(500) * 5,
        'feature3': np.random.randn(500),
        'image_link': [f'link_{i}' for i in range(500)],
        'sample_id': range(500)
    })

X = data.drop(columns=["price", "image_link", "sample_id"])
y = data["price"].values

# =========================================================
# Preprocessing
# =========================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_log = np.log1p(y)

# =========================================================
# LightGBM parameters for Optuna
# =========================================================
params = {
    "objective": "regression_l1",
    "metric": "l1",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "seed": 42,
    "n_jobs": -1,
}

# =========================================================
# Prepare data + cross-validation
# =========================================================
dtrain = lgb.Dataset(X_scaled, label=y_log)
rkf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# =========================================================
# Optuna Study
# =========================================================
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_tuner = optuna.create_study(direction='minimize', study_name='lgbm_tuner_v4')

# =========================================================
# Run LightGBM TunerCV
# =========================================================
tuner = LightGBMTunerCV(
    params,
    dtrain,
    folds=rkf,
    study=study_tuner,
    # verbose_eval=False, # <-- THIS LINE WAS REMOVED
    early_stopping_rounds=100,
    time_budget=7200,
    seed=42,
    num_boost_round=10000,
)

print("🚀 Running Optuna LightGBM tuning...")
tuner.run()

# =========================================================
# Best Parameters
# =========================================================
best_params = tuner.best_params
print("\n==========================")
print("🏆 Best parameters found by Optuna:")
for k, v in best_params.items():
    print(f"{k}: {v}")
print("==========================")

# =========================================================
# Final Training using best parameters
# =========================================================
final_model = lgb.train(
    best_params,
    dtrain,
)

# =========================================================
# Evaluate with a more reliable metric
# =========================================================
best_cv_score = study_tuner.best_value
print(f"📊 Best Cross-Validated MAE (on log-price): {best_cv_score:.6f}")
print("==========================")

y_pred_log_insample = final_model.predict(X_scaled)
y_pred_insample = np.expm1(y_pred_log_insample)
smape_insample = smape(y, y_pred_insample)
print(f"⚠️ In-Sample SMAPE (for reference only): {smape_insample:.2f}%")
print("==========================")

TypeError: LightGBMTunerCV.__init__() got an unexpected keyword argument 'early_stopping_rounds'

In [13]:
pip install optuna-integration[lightgbm]

Defaulting to user installation because normal site-packages is not writeable

  Attempting uninstall: joblib

    Found existing installation: joblib 1.1.1

    Uninstalling joblib-1.1.1:

      Successfully uninstalled joblib-1.1.1

   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   -------------------- ------------------- 1/2 [optuna-integration]
   -------------------- ------------------- 1/2 [optuna-integration]
   -------------------- ------------------- 1/2 [optuna-integration]
   -------------------- ------------------- 1/2 [optuna-integration]
   -------------------- ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.5.2 which is incompatible.
pandas-profiling 3.2.0 requires visions[type_image_path]==0.7.4, but you have visions 0.8.1 which is incompatible.


In [24]:

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import optuna
from optuna.pruners import MedianPruner

# Create sample dataset
X, y = make_classification(n_samples=10000, n_features=20, n_informative=15,
                           n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dataset shape:", X_train.shape)
print("="*80)

# ============================================================================
# METHOD 1: GRID SEARCH
# ============================================================================
print("\n1. GRID SEARCH")
print("-"*80)

param_grid = {
    'num_leaves': [31, 50, 100],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_data_in_leaf': [20, 50]
}

best_score = 0
best_params = {}

for num_leaves in param_grid['num_leaves']:
    for max_depth in param_grid['max_depth']:
        for lr in param_grid['learning_rate']:
            for min_leaf in param_grid['min_data_in_leaf']:
                params = {
                    'objective': 'binary',
                    'metric': 'binary_logloss',
                    'num_leaves': num_leaves,
                    'max_depth': max_depth,
                    'learning_rate': lr,
                    'min_data_in_leaf': min_leaf,
                    'verbose': -1
                }
                
                # Train with early stopping
                train_data = lgb.Dataset(X_train, label=y_train)
                model = lgb.train(
                    params,
                    train_data,
                    num_boost_round=500,
                    valid_sets=[train_data],
                    callbacks=[lgb.early_stopping(50)]
                )
                
                y_pred = model.predict(X_test) > 0.5
                score = accuracy_score(y_test, y_pred)
                
                if score > best_score:
                    best_score = score
                    best_params = params.copy()
                    print(f"New best score: {score:.4f}")
                    print(f"  Params: leaves={num_leaves}, depth={max_depth}, lr={lr}, min_leaf={min_leaf}")

print(f"\nBest Grid Search Score: {best_score:.4f}")
print(f"Best Params: {best_params}")

# ============================================================================
# METHOD 2: RANDOM SEARCH
# ============================================================================
print("\n\n2. RANDOM SEARCH")
print("-"*80)

np.random.seed(42)
best_score_random = 0
best_params_random = {}

for trial in range(20):  # 20 random trials
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': np.random.randint(20, 150),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.2),
        'min_data_in_leaf': np.random.randint(10, 100),
        'lambda_l1': np.random.uniform(0, 50),
        'lambda_l2': np.random.uniform(0, 50),
        'feature_fraction': np.random.uniform(0.5, 1.0),
        'bagging_fraction': np.random.uniform(0.5, 1.0),
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[train_data],
        callbacks=[lgb.early_stopping(50)]
    )
    
    y_pred = model.predict(X_test) > 0.5
    score = accuracy_score(y_test, y_pred)
    
    if score > best_score_random:
        best_score_random = score
        best_params_random = params.copy()
        print(f"Trial {trial+1}: New best score: {score:.4f}")

print(f"\nBest Random Search Score: {best_score_random:.4f}")

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
from optuna.pruners import MedianPruner

# =========================================================
# Preprocessing
# =========================================================
# X = data.drop(columns=["price", "image_link", "sample_id"])
# y = data["price"].values

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_val_scaled = scaler.transform(X_val_raw)

# Log-transform target
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print("Dataset shape:", X_train_scaled.shape)
print("Target range (original):", y_train.min(), "-", y_train.max())
print("="*80)

# ============================================================================
# METHOD 1: RANDOM SEARCH (REGRESSION)
# ============================================================================
print("\n1. RANDOM SEARCH")
print("-"*80)

np.random.seed(42)
best_score_random = float('inf')
best_params_random = {}
best_model_random = None

for trial in range(20):  # 20 random trials
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': np.random.randint(20, 150),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.2),
        'min_data_in_leaf': np.random.randint(10, 100),
        'lambda_l1': np.random.uniform(0, 50),
        'lambda_l2': np.random.uniform(0, 50),
        'feature_fraction': np.random.uniform(0.5, 1.0),
        'bagging_fraction': np.random.uniform(0.5, 1.0),
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train_scaled, label=y_train_log)
    val_data = lgb.Dataset(X_val_scaled, label=y_val_log, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(-1)]
    )
    
    # Predict and inverse transform
    y_val_pred_log = model.predict(X_val_scaled)
    y_val_pred = np.expm1(y_val_pred_log)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    if rmse < best_score_random:
        best_score_random = rmse
        best_params_random = params.copy()
        best_model_random = model
        print(f"Trial {trial+1}: New best RMSE: {rmse:.4f}")
        print(f"  R² Score: {r2_score(y_val, y_val_pred):.4f}")
        print(f"  MAE: {mean_absolute_error(y_val, y_val_pred):.4f}")

print(f"\nBest Random Search RMSE: {best_score_random:.4f}")


Dataset shape: (8000, 20)

1. GRID SEARCH
--------------------------------------------------------------------------------
New best score: 0.9250
  Params: leaves=31, depth=5, lr=0.01, min_leaf=20
New best score: 0.9590
  Params: leaves=31, depth=5, lr=0.05, min_leaf=20
New best score: 0.9640
  Params: leaves=31, depth=5, lr=0.1, min_leaf=20
New best score: 0.9650
  Params: leaves=31, depth=5, lr=0.1, min_leaf=50
New best score: 0.9655
  Params: leaves=31, depth=10, lr=0.1, min_leaf=20
New best score: 0.9700
  Params: leaves=31, depth=10, lr=0.1, min_leaf=50
New best score: 0.9720
  Params: leaves=50, depth=15, lr=0.1, min_leaf=20

Best Grid Search Score: 0.9720
Best Params: {'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 50, 'max_depth': 15, 'learning_rate': 0.1, 'min_data_in_leaf': 20, 'verbose': -1}


2. RANDOM SEARCH
--------------------------------------------------------------------------------
Trial 1: New best score: 0.9140
Trial 2: New best score: 0.9535
Tria

In [ ]:
lgb_model = lgb.LGBMRegressor(
    objective="regression",
    learning_rate=0.05,
    n_estimators=250,
    num_leaves=32,
    feature_fraction=0.8,
    bagging_fraction=0.85,
    bagging_freq=3,
    random_state=42,
    n_jobs=-1
)
Final MAE: 15.0662
Final SMAPE: 65.98%

In [ ]:

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import optuna
from optuna.pruners import MedianPruner


print("Dataset shape:", X_train.shape)
print("="*80)

# ============================================================================
# METHOD 1: GRID SEARCH
# ============================================================================
print("\n1. GRID SEARCH")
print("-"*80)

param_grid = {
    'num_leaves': [31, 50, 100],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_data_in_leaf': [20, 50]
}

best_score = 0
best_params = {}

for num_leaves in param_grid['num_leaves']:
    for max_depth in param_grid['max_depth']:
        for lr in param_grid['learning_rate']:
            for min_leaf in param_grid['min_data_in_leaf']:
                params = {
                    'objective': 'binary',
                    'metric': 'binary_logloss',
                    'num_leaves': num_leaves,
                    'max_depth': max_depth,
                    'learning_rate': lr,
                    'min_data_in_leaf': min_leaf,
                    'verbose': -1
                }
                
                # Train with early stopping
                train_data = lgb.Dataset(X_train, label=y_train)
                model = lgb.train(
                    params,
                    train_data,
                    num_boost_round=500,
                    valid_sets=[train_data],
                    callbacks=[lgb.early_stopping(50)]
                )
                
                y_pred = model.predict(X_test) > 0.5
                score = accuracy_score(y_test, y_pred)
                
                if score > best_score:
                    best_score = score
                    best_params = params.copy()
                    print(f"New best score: {score:.4f}")
                    print(f"  Params: leaves={num_leaves}, depth={max_depth}, lr={lr}, min_leaf={min_leaf}")

print(f"\nBest Grid Search Score: {best_score:.4f}")
print(f"Best Params: {best_params}")

# ============================================================================
# METHOD 2: RANDOM SEARCH
# ============================================================================
print("\n\n2. RANDOM SEARCH")
print("-"*80)

np.random.seed(42)
best_score_random = 0
best_params_random = {}

for trial in range(20):  # 20 random trials
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': np.random.randint(20, 150),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.2),
        'min_data_in_leaf': np.random.randint(10, 100),
        'lambda_l1': np.random.uniform(0, 50),
        'lambda_l2': np.random.uniform(0, 50),
        'feature_fraction': np.random.uniform(0.5, 1.0),
        'bagging_fraction': np.random.uniform(0.5, 1.0),
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[train_data],
        callbacks=[lgb.early_stopping(50)]
    )
    
    y_pred = model.predict(X_test) > 0.5
    score = accuracy_score(y_test, y_pred)
    
    if score > best_score_random:
        best_score_random = score
        best_params_random = params.copy()
        print(f"Trial {trial+1}: New best score: {score:.4f}")

print(f"\nBest Random Search Score: {best_score_random:.4f}")


import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
from optuna.pruners import MedianPruner

# =========================================================
# Preprocessing
# =========================================================
# X = data.drop(columns=["price", "image_link", "sample_id"])
# y = data["price"].values
data = pd.read_csv("final_pca_whole_train_data.csv")
X = data.drop(columns=["price", "image_link", "sample_id"])
y = data["price"].values
X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_val_scaled = scaler.transform(X_val_raw)

# Log-transform target
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print("Dataset shape:", X_train_scaled.shape)
print("Target range (original):", y_train.min(), "-", y_train.max())
print("="*80)

# ============================================================================
# METHOD 1: RANDOM SEARCH (REGRESSION)
# ============================================================================
print("\n1. RANDOM SEARCH")
print("-"*80)

np.random.seed(42)
best_score_random = float('inf')
best_params_random = {}
best_model_random = None

for trial in range(20):  # 20 random trials
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': np.random.randint(20, 150),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.2),
        'min_data_in_leaf': np.random.randint(10, 100),
        'lambda_l1': np.random.uniform(0, 50),
        'lambda_l2': np.random.uniform(0, 50),
        'feature_fraction': np.random.uniform(0.5, 1.0),
        'bagging_fraction': np.random.uniform(0.5, 1.0),
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train_scaled, label=y_train_log)
    val_data = lgb.Dataset(X_val_scaled, label=y_val_log, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(-1)]
    )
    
    # Predict and inverse transform
    y_val_pred_log = model.predict(X_val_scaled)
    y_val_pred = np.expm1(y_val_pred_log)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    if rmse < best_score_random:
        best_score_random = rmse
        best_params_random = params.copy()
        best_model_random = model
        print(f"Trial {trial+1}: New best RMSE: {rmse:.4f}")
        print(f"  R² Score: {r2_score(y_val, y_val_pred):.4f}")
        print(f"  MAE: {mean_absolute_error(y_val, y_val_pred):.4f}")

print(f"\nBest Random Search RMSE: {best_score_random:.4f}")

# ============================================================================
# METHOD 2: BAYESIAN OPTIMIZATION WITH OPTUNA
# ============================================================================
print("\n\n2. BAYESIAN OPTIMIZATION (OPTUNA)")
print("-"*80)


import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# =========================================================
# Preprocessing
# =========================================================
# X = data.drop(columns=["price", "image_link", "sample_id"])
# y = data["price"].values

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_val_scaled = scaler.transform(X_val_raw)

# Log-transform target
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print("Dataset shape:", X_train_scaled.shape)
print("Target range (original):", y_train.min(), "-", y_train.max())
print("="*80)

# ============================================================================
# RANDOM SEARCH
# ============================================================================
print("\nRANDOM SEARCH")
print("-"*80)

np.random.seed(42)
best_score_random = float('inf')
best_params_random = {}
best_model_random = None

for trial in range(20):  # 20 random trials
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': np.random.randint(20, 150),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.2),
        'min_data_in_leaf': np.random.randint(10, 100),
        'lambda_l1': np.random.uniform(0, 50),
        'lambda_l2': np.random.uniform(0, 50),
        'feature_fraction': np.random.uniform(0.5, 1.0),
        'bagging_fraction': np.random.uniform(0.5, 1.0),
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train_scaled, label=y_train_log)
    val_data = lgb.Dataset(X_val_scaled, label=y_val_log, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(-1)]
    )
    
    # Predict and inverse transform
    y_val_pred_log = model.predict(X_val_scaled)
    y_val_pred = np.expm1(y_val_pred_log)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    if rmse < best_score_random:
        best_score_random = rmse
        best_params_random = params.copy()
        best_model_random = model
        print(f"Trial {trial+1}: New best RMSE: {rmse:.4f}")
        print(f"  R² Score: {r2_score(y_val, y_val_pred):.4f}")
        print(f"  MAE: {mean_absolute_error(y_val, y_val_pred):.4f}")

print(f"\n{'='*80}")
print(f"BEST MODEL FROM RANDOM SEARCH")
print(f"{'='*80}")
print(f"Best RMSE: {best_score_random:.4f}")
print(f"\nBest Hyperparameters:")
for key, value in best_params_random.items():
    print(f"  {key}: {value}")

# ============================================================================
# EVALUATE BEST MODEL
# ============================================================================
print(f"\n{'='*80}")
print(f"FINAL EVALUATION")
print(f"{'='*80}")

y_val_pred_log = best_model_random.predict(X_val_scaled)
y_val_pred = np.expm1(y_val_pred_log)

final_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
final_mae = mean_absolute_error(y_val, y_val_pred)
final_r2 = r2_score(y_val, y_val_pred)

print(f"RMSE: {final_rmse:.4f}")
print(f"MAE: {final_mae:.4f}")
print(f"R² Score: {final_r2:.4f}")

# ============================================================================
# SAVE AND USE THE BEST MODEL
# ============================================================================
print(f"\n{'='*80}")
print(f"SAVING BEST MODEL")
print(f"{'='*80}")

# Save model
pickle.dump(best_model_random, open('best_model.pkl', 'wb'))
print("✓ Model saved to 'best_model.pkl'")

# Save scaler
pickle.dump(scaler, open('scaler.pkl', 'wb'))
print("✓ Scaler saved to 'scaler.pkl'")

Dataset shape: (8000, 20)

1. GRID SEARCH
--------------------------------------------------------------------------------
New best score: 0.9250
  Params: leaves=31, depth=5, lr=0.01, min_leaf=20
New best score: 0.9590
  Params: leaves=31, depth=5, lr=0.05, min_leaf=20
New best score: 0.9640
  Params: leaves=31, depth=5, lr=0.1, min_leaf=20
New best score: 0.9650
  Params: leaves=31, depth=5, lr=0.1, min_leaf=50
New best score: 0.9655
  Params: leaves=31, depth=10, lr=0.1, min_leaf=20
New best score: 0.9700
  Params: leaves=31, depth=10, lr=0.1, min_leaf=50


In [31]:
# =========================================================
# Load test data
# =========================================================
import pandas as pd
test_data = pd.read_csv("final_pca_whole_test_data.csv")  # 👈 Replace with your test dataset path
sample_id = test_data['sample_id']

# Drop unnecessary columns that are not part of training
drop_cols = ['sample_id','image_link']  # ensure you don’t have this in test
test_data = test_data.drop(columns=[c for c in drop_cols if c in test_data.columns], errors='ignore')

# =========================================================
# Align test columns to match training columns
# =========================================================
# missing_cols = set(X_train_raw.columns) - set(test_data.columns)
# for col in missing_cols:
#     test_data[col] = 0  # fill missing columns with 0

# # Ensure the same column order as training
# test_data = test_data[X_train_raw.columns]

# =========================================================
# Scale test data (same scaler as training)
# =========================================================
X_test_scaled = scaler.transform(test_data)

# =========================================================
# Predict on test data
# =========================================================
y_test_pred_log = best_model_random.predict(X_test_scaled)
y_test_pred = np.expm1(y_test_pred_log)  # reverse log-transform

# =========================================================
# Create submission file
# =========================================================
# If your test file has a sample_id column, keep it

submission = pd.DataFrame({
    'sample_id': sample_id,
    'price': y_test_pred
    })

# Save submission
submission.to_csv("submissionn.csv", index=False)
print("✅ Predictions saved to submission.csv")

ValueError: X has 603 features, but StandardScaler is expecting 20 features as input.